In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
# Reading the entire csv with all the sheets
big_data = pd.ExcelFile("data/raw/MM_for_nov17_aug19.xlsx")

In [4]:
# Sheet names
months = big_data.sheet_names

In [5]:
# List of columns that contains prices and are present for all months
columns_required = ["region", "q_gov", "q_district", "q_sbd",  "q_town",
                    "q_bread_price_per_8pieces", "q_bulgur_price_per_kilo", "q_chicken_price_per_kilo",
                    "q_eggs_price_per_30eggs", "q_potatoes_price_per_kilo", "q_tomatoes_price_per_kilo", 
                    "q_ghee_price_per_kilo", "q_oil_price_per_litre", "q_rlentils_price_per_kilo", 
                    "q_rice_price_per_kilo","q_salt_price_per_500g", "q_sugar_price_per_kilo", 
                    "q_tomatop_price_per_kilo", "q_onions_price_per_kilo", "q_cucumbers_price_per_kilo",
                    "q_isoap_price_per_piece", "q_lsoap_price_per_kilo", "q_dsoap_price_per_litre",
                    "q_spads_price_per_10pads", "q_toothp_price_per_100g", "q_sgas_price",
                    "q_mrkaz_price", "q_water_price_per_litre", "q_data_price_per_gb",
                    "q_rgpetrol_price", "q_mrpetrol_price", "q_rgdiesel_price", "q_mrdiesel_price",
                    "q_xrate_usdsyp_buy", "q_xrate_usdsyp_sell", "q_xrate_trysyp_buy",
                    "q_xrate_trysyp_sell", "q_xrate_jodsyp_buy", "q_xrate_jodsyp_sell",]

In [7]:
len(columns_required)

39

In [6]:
months

['Nov 17',
 'Dec 17',
 'Jan 18',
 'Feb 18',
 'March 18',
 'April 18',
 'May 18',
 'June 18',
 'July 18',
 'Aug18',
 'Sep 18',
 'Oct 18',
 'Nov 18',
 'Dec 18',
 'Jan 19',
 'Feb 19',
 'March 19',
 'April 19',
 'May 19',
 'June 19',
 'July 19',
 'Aug 19']

In [8]:
# Creating an empty dataframe with all the required columns
df = pd.DataFrame(columns = columns_required)

In [9]:
# Creating a list of dataframe containing only the required columns for each month
df_months = []
for mon in months:
    n = len(big_data.parse(mon))
    month_col = [mon]*n
    df_mon = big_data.parse(mon)
    df_mon = df_mon[columns_required]
    df_mon['Month'] = month_col # adding column "Month" with repeated Month name
    df_months.append(df_mon)

In [34]:
len(big_data.parse('June 18'))

2375

In [56]:
all_prices_df = pd.concat(df_months) # Merging all months into one

In [57]:
all_prices_df.reset_index(drop=True, inplace=True)

In [58]:
name_cols = ["admin1Name_en", "admin2Name_en", "admin3Name_en", "admin4Name_en"]
for col in name_cols:
    all_prices_df[col] = ""

## Correcting town mapping to sub_districts, districts and govs

In [126]:
locations = pd.read_excel('data/raw/UNOCHA_pcodes_nocamps.xlsx', sheet_name = 'admin4')
all_towns = list(locations['admin4Pcode'])

In [127]:
for town in all_towns:
    # creating list of indexes with a particular town name
    idx1 = all_prices_df[all_prices_df['q_town'] == town].index
    n = len(idx1)
    # Certain towns may not be present in our data
    if n>0:
        # Assigning correct sbd, district and gov codes to the corresponding town
        idx2 = locations[locations['admin4Pcode'] == town].index[0]
        all_prices_df.loc[idx1, 'q_sbd'] = locations.loc[idx2, 'admin3Pcode']
        all_prices_df.loc[idx1, 'q_district'] = locations.loc[idx2, 'admin2Pcode']
        all_prices_df.loc[idx1, 'q_gov'] = locations.loc[idx2, 'admin1Pcode']
        # Adding town name
        all_prices_df.loc[idx1, 'admin4Name_en'] = locations.loc[idx2, 'admin4Name_en']

In [43]:
all_prices_df

,region,q_gov,q_district,q_sbd,q_town,q_bread_price_per_8pieces,q_bulgur_price_per_kilo,q_chicken_price_per_kilo,q_eggs_price_per_30eggs,q_potatoes_price_per_kilo,...,q_xrate_usdsyp_sell,q_xrate_trysyp_buy,q_xrate_trysyp_sell,q_xrate_jodsyp_buy,q_xrate_jodsyp_sell,Month,admin1Name_en,admin2Name_en,admin3Name_en,admin4Name_en
0,besieged,SY03,SY0301,SY030105,C2314,1644.44,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
1,besieged,SY03,SY0301,SY030105,C2314,1644.44,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
2,besieged,SY03,SY0301,SY030105,C2314,2044.44,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
3,besieged,SY03,SY0301,SY030105,C2314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
4,besieged,SY03,SY0301,SY030105,C2314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
5,besieged,SY03,SY0301,SY030105,C2314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
6,besieged,SY03,SY0301,SY030105,C2314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
7,besieged,SY03,SY0301,SY030105,C2314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
8,besieged,SY03,SY0301,SY030105,C2314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Saqba
9,besieged,SY03,SY0302,SY030200,C2338,1644.44,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Nov 17,,,,Duma


In [128]:
## Adding names of sbd, districts and govs
all_sbd = list(locations['admin3Pcode'])
all_sbd = np.unique(all_sbd)
for sbd in all_sbd:
    idx1 = all_prices_df[all_prices_df['q_sbd'] == sbd].index
    n = len(idx1)
    # Certain sbds may not be present in our data
    if n>0:
        idx2 = locations[locations['admin3Pcode'] == sbd].index[0]
        all_prices_df.loc[idx1, 'admin1Name_en'] = locations.loc[idx2, 'admin1Name_en']
        all_prices_df.loc[idx1, 'admin2Name_en'] = locations.loc[idx2, 'admin2Name_en']
        all_prices_df.loc[idx1, 'admin3Name_en'] = locations.loc[idx2, 'admin3Name_en']

In [129]:
# Removing sbd that do not exists!
sbd_in_data = set(np.unique(list(all_prices_df["q_sbd"])))
sbd_nonexistent = sbd_in_data.difference(set(all_sbd))

sbd_nonexistent = sbd_nonexistent.difference({'nan'})

for sbd in sbd_nonexistent:
    idx = all_prices_df[all_prices_df['q_sbd'] == sbd].index
    print(idx)
    all_prices_df.drop(idx, inplace= True)

In [130]:
# Reordering columns
col_order = ["Month", "region", "q_gov", "admin1Name_en", "q_district", "admin2Name_en", "q_sbd",
             "admin3Name_en", "q_town", "admin4Name_en",
            "q_bread_price_per_8pieces", "q_bulgur_price_per_kilo", "q_chicken_price_per_kilo",
            "q_eggs_price_per_30eggs", "q_potatoes_price_per_kilo", "q_tomatoes_price_per_kilo", 
            "q_ghee_price_per_kilo", "q_oil_price_per_litre", "q_rlentils_price_per_kilo", 
            "q_rice_price_per_kilo","q_salt_price_per_500g", "q_sugar_price_per_kilo", 
            "q_tomatop_price_per_kilo", "q_onions_price_per_kilo", "q_cucumbers_price_per_kilo",
            "q_isoap_price_per_piece", "q_lsoap_price_per_kilo", "q_dsoap_price_per_litre",
            "q_spads_price_per_10pads", "q_toothp_price_per_100g", "q_sgas_price",
            "q_mrkaz_price", "q_water_price_per_litre", "q_data_price_per_gb",
            "q_rgpetrol_price", "q_mrpetrol_price", "q_rgdiesel_price", "q_mrdiesel_price",
            "q_xrate_usdsyp_buy", "q_xrate_usdsyp_sell", "q_xrate_trysyp_buy",
            "q_xrate_trysyp_sell", "q_xrate_jodsyp_buy", "q_xrate_jodsyp_sell",]

In [171]:
mydf = all_prices_df[col_order]
mydf.head()

,Month,region,q_gov,admin1Name_en,q_district,admin2Name_en,q_sbd,admin3Name_en,q_town,admin4Name_en,...,q_rgpetrol_price,q_mrpetrol_price,q_rgdiesel_price,q_mrdiesel_price,q_xrate_usdsyp_buy,q_xrate_usdsyp_sell,q_xrate_trysyp_buy,q_xrate_trysyp_sell,q_xrate_jodsyp_buy,q_xrate_jodsyp_sell
0,Nov 17,besieged,SY03,Rural Damascus,SY0301,Rural Damascus,SY030105,Kafr Batna,C2314,Saqba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nov 17,besieged,SY03,Rural Damascus,SY0301,Rural Damascus,SY030105,Kafr Batna,C2314,Saqba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nov 17,besieged,SY03,Rural Damascus,SY0301,Rural Damascus,SY030105,Kafr Batna,C2314,Saqba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nov 17,besieged,SY03,Rural Damascus,SY0301,Rural Damascus,SY030105,Kafr Batna,C2314,Saqba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nov 17,besieged,SY03,Rural Damascus,SY0301,Rural Damascus,SY030105,Kafr Batna,C2314,Saqba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
# Removing entries of sbd that do not exist in their location mapping database (basically incorrect entry)
sbd_names = mydf['admin3Name_en'].values
sbd_names_cleaned = []
for i in sbd_names:
    s = re.sub('\W+', '', i)
    sbd_names_cleaned.append(s)

mydf['admin3Name_en'] = sbd_names_cleaned

In [174]:
# Converting empty string into NaN
all_cols = mydf.columns
for col in all_cols:
    if ("price" in col) | ("rate" in col):
        mydf[col] = list(mydf[col])

In [178]:
mydf.to_csv("data/processed/aggregated_monthly_1.csv")

In [ ]:
# # Converting empty string into NaN
# n, d = all_prices_df.shape
# for i in range(n):
#     for j in range(d-6): # Removing the text columns
#         if type(all_prices_df.iloc[i,j])== str:
#             all_prices_df.iloc[i,j] = float('NaN')